# Setup Instructions
1. Upload '25hz_down_sampled.csv' to the sample_data directory
2. Upgrade the ipython kernel to support asyncio (note you will see an error and a warning; don't worry, for what we are doing all is well)
3. Restart the runtime using the button that comes up in the pip output
4. Install the azure-iot-device package

In [ ]:
%pip install ipython ipykernel --upgrade

In [ ]:
%pip install azure-iot-device

# Azure IoT Hub Device Simulator

Azure documentation: https://docs.microsoft.com/en-us/samples/azure-samples/azure-iot-samples-python/azure-iot-samples-for-python/


## IoT Hub Message Structure

The content of the message we send is referred to as the 'payload.'  While IoT Hub doesn't care what the message structure is, SAS Intelligent Monitoring does.  The message structure must be consistent with the definition in our SAS Intelligent Monitoring configuration.

Our data is defined as: 
<ul>
    <li>varId3 = {Temperature in Celsius}</li>
    <li>varId4 = {X-motion}</li>
    <li>varId5 = {Y-motion}</li> 
    <li>varId6 = {Z-motion}</li>

```
{
    "telemetryDataList": [
      {
        "varId": "3",
        "devId": "202801",
        "dateTime": "Jan 27, 2022 06:44:12 PM",
        "value": "88.3"
      },
      {
        "varId": "4",
        "devId": "202801",
        "dateTime": "Jan 27, 2022 06:44:12 PM",
        "value": "-0.17123"
      },
      {
        "varId": "5",
        "devId": "202801",
        "dateTime": "Jan 27, 2022 06:44:12 PM",
        "value": "0.04351"
      }
      {
        "varId": "6",
        "devId": "202801",
        "dateTime": "Jan 27, 2022 06:44:12 PM",
        "value": "-0.58459"
      }
    ],
  }

```

# The Code

In [1]:
import os
import asyncio
from azure.iot.device.aio import IoTHubDeviceClient
import json
from time import sleep
from datetime import datetime

import csv
import time
import pytz
import random

# In a live environment, this should be loaded from an environment variable, not code
CONN_STR_202801=''

### Configure your virtual device
1. Assign your device ID to  ```MY_DEVICE_ID``` 
2. Change ```CONN_STRING_202801``` to end with your device ID
3. Double check that you uploaded the data file ```25hz_down_sampled.csv``` to the ```sample_data``` directory

In [2]:
DATA_FILE='sample_data/25hz_down_sampled.csv'

MY_DEVICE_ID='20801'
MY_CONN_STRING = CONN_STR_202801

In [3]:
#Utilities

def UTC_now_string(decimal_digits):
    decimal_places = -6+decimal_digits if decimal_digits <=6 and decimal_digits >= 1 else -7
    return( datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:decimal_places] )


def build_json_msg(dataVals, devId, dateTime=None):
    dt = UTC_now_string(0) if dateTime is None else dateTime
    dataList=[]
    for n,val in enumerate(dataVals):
        dataList.append({
                            "varId" : str(n+2),
                            "devId" : devId,
                            "dateTime" : dt,
                            "value" : val,
                        }
        )
    return( {"telemetrydataList" : dataList } )

### The IoT Device Simulator class

In [6]:
class IoT_Device_Sim():
    csv_file=None
    csv_reader=None
    file_path=None
    _device_Id=None
    _conn_str=None
    _device_client=None
    _force_alerts=False
    
    def __init__(self, device_Id, connection_string, path_to_data_file=None, ):
        self._device_Id=device_Id
        self._conn_str=connection_string
        self.file_path = path_to_data_file
        self.open_data_file()
        
    # Azure IoTHub Device Client calls
    async def connect(self):
        if self._conn_str is not None:
            self._device_client = IoTHubDeviceClient.create_from_connection_string(self._conn_str)
        await self._device_client.connect()
        print("Device Simulator {a} Connected".format(a=self._device_Id))
            
    async def send_message(self, msg):
        print("Sending message: " + msg)
        await self._device_client.send_message(msg)
                                 
    async def disconnect(self):
        await self._device_client.disconnect()    
    
    
    # For getting device data from a file
    def open_data_file(self):
        if self.file_path is not None:
            try:
                self.csv_file=open(self.file_path, 'r')
            except OSError as err:
                print(err)
                self.csv_file=None
            else:
                print( 'Using ' + self.file_path + ' as data source')
                self.csv_reader=(csv.reader(self.csv_file))
            
    def get_reading(self):
        if self.csv_reader is None:
            return( None )
        else:
            row = next(self.csv_reader)
            ts, date_time, _, _, _, temp, _, x, y, z = row
            if self._force_alerts:
              temp=random.randrange(900,980,1) / 10
            #print( ts, temp, x, y, z )
            data = self.build_SAS_json( datetime.timestamp(datetime.now()), UTC_now_string(0), temp, x, y, z, self._device_Id )
            return( data )
        
    def build_SAS_json( self,ts, date_time, temp, x, y, z, device_Id ):
        t_list=[]
        vals = [temp, x, y, z]           
        for i in range(1,5):
            t_list.append( {'devId' : device_Id,
                            'varId' : str(i+2),
                            'dateTime' : date_time,
                            'value' : vals[i-1]
                           }
                         )
        json_str = json.dumps({'telemetryDataList' : t_list})
        return( json_str )
    
    async def run(self, duration_seconds=120, wait_interval_seconds=30):
        for i in range(0, duration_seconds+1, wait_interval_seconds):
            time.sleep(wait_interval_seconds)
            reading = (self.get_reading())
            #print(reading)
            if reading is None:
                self.reset()
                pass
            else:
                await self.send_message(reading)
    
    def force_alerts(self, set=True):
      self._force_alerts=set

    def reset(self):
        if self.csv_file is not None:
            self.csv_file.seek(0)
    
    async def stop(self):
        await self.disconnect()

### Create, connect, and run the device simulator
To force alerts to be generated, uncomment the second line in the cell below.

In [ ]:
my_device=IoT_Device_Sim(MY_DEVICE_ID,MY_CONN_STRING,DATA_FILE)
#my_device.force_alerts(set=True)
await my_device.connect()

await my_device.run(60, 10)

In [ ]:
await my_device.run(20, 5)

### Shut down the device sumulator
Housekeeping to release resources used by the simulator

In [8]:
await my_device.stop()

